## IMPORT

In [3]:
!pip install --upgrade pip
!pip install gradio
!pip install --upgrade transformers sentencepiece datasets[audio]
!pip install lmqg
!pip install --upgrade evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install rouge_score

In [7]:
import gradio as gr
import librosa
import numpy as np
import lmqg
import evaluate
import rouge_score
from pprint import pprint
from lmqg import TransformersQG
from transformers import pipeline

In [32]:
# initialize model
model = TransformersQG(language='en', model='lmqg/t5-base-squad-qg-ae')

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2991: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [19]:
context = "Artificial Intelligence (AI) represents the forefront of computer science and cognitive science. It is an interdisciplinary field that focuses on creating systems and machines capable of performing tasks that typically require human intelligence. These systems learn from data, identify patterns, and make predictions or decisions based on these patterns. Applications of AI are widespread across various sectors, from healthcare to autonomous driving, finance to industrial production. AI utilizes a wide range of techniques, including machine learning, artificial neural networks, and natural language processing."

# model prediction
question_answer = model.generate_qa(context)  # Assuming model has a generate_qa method

# Separate questions and answers
questions = []
answers = []
for question, answer in question_answer:
  questions.append(question)
  answers.append(answer)

# Print questions and answers separately
print("Questions:")
for question in questions:
  print(question)

print("\nAnswers:")
for answer in answers:
  print(answer)


100%|██████████| 5/5 [00:00<00:00, 171.88it/s]


Questions:
What is the name of the interdisciplinary field that focuses on creating systems and machines capable of performing tasks that typically require human intelligence?
What does AI focus on?
What do AI systems make based on patterns?
What is an example of a sector that uses AI?
What are some techniques that AI uses?

Answers:
Artificial Intelligence
creating systems and machines capable of performing tasks
predictions or decisions
healthcare
machine learning, artificial neural networks, and natural language processing.


In [33]:
# Modello per la gestione delle domande e delle risposte
class Model:
    def __init__(self, questions, answers):
        self.questions = questions
        self.answers = answers
        self.score = 0

    def check_answer(self, user_answer, correct_answer):
        rouge = evaluate.load('rouge')  # Carica l'indice ROUGE
        # List containing the reference summary
        references = [correct_answer]
        # List containing the user's answer
        predictions = [user_answer]

        # Calcolo della somiglianza tra risposta utente e risposta corretta
        rouge_result = rouge.compute(predictions=predictions, references=references)

        # Estrazione del valore di somiglianza per ROUGE-1
        rouge_score = rouge_result['rouge1']  # Similitudine ROUGE-1

        # Restituzione del valore di somiglianza
        return rouge_score

# Vista per l'interfaccia con l'utente
class View:
    def ask_question(self, question):
        return input(question)

    def show_score(self, score):
        print(f"Your current score is: {score}")

# Controller per la gestione del modello e della vista
class Controller:
    def __init__(self, model, view):
        self.model = model
        self.view = view

    def run(self):
        for question, answer in zip(self.model.questions, self.model.answers):
            user_answer = self.view.ask_question(question)
            score = self.model.check_answer(user_answer, answer)
            self.view.show_score(score)

# Dati delle domande e delle risposte
questions = ["What's the capital of France?", "What's the capital of Italy?"]
answers = ["Paris", "Rome"]

# Inizializzazione del gioco
model = Model(questions, answers)
view = View()
controller = Controller(model, view)

# Avvia il gioco
controller.run()

What's the capital of France?va
Your current score is: 0.0
What's the capital of Italy?va
Your current score is: 0.0


In [8]:
model = TransformersQG(language='en', model='lmqg/t5-base-squad-qg-ae')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2991: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [9]:
MODEL_NAME = "openai/whisper-small"
BATCH_SIZE = 8

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
)


# Copied from https://github.com/openai/whisper/blob/c09a7ae299c4c34c5839a76380ae407e7d785914/whisper/utils.py#L50
def format_timestamp(seconds: float, always_include_hours: bool = False, decimal_marker: str = "."):
    if seconds is not None:
        milliseconds = round(seconds * 1000.0)

        hours = milliseconds // 3_600_000
        milliseconds -= hours * 3_600_000

        minutes = milliseconds // 60_000
        milliseconds -= minutes * 60_000

        seconds = milliseconds // 1_000
        milliseconds -= seconds * 1_000

        hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
        return f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"
    else:
        # we have a malformed timestamp so just return it as is
        return seconds


def transcribe(file):
    outputs = pipe(file, batch_size=BATCH_SIZE)
    text = outputs["text"]
    return text

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
import gradio as gr
import logging

questions = []
current_question = 0
answers = []
score = 0
sentiment_label = None

logging.basicConfig(level=logging.DEBUG)

def load_data(input_data):
    global questions, answers, score, current_question  # Use the global variables
    try:
        context = input_data
        # model prediction
        question_answer = model.generate_qa(context)  # Assuming model has a generate_qa method
        #question_answer = [(i, i) for i in range(1,5)]
        # Separate questions and answers
        questions = []
        answers = []
        score = 0
        current_question = 0
        for question, answer in question_answer:
            questions.append(question)
            answers.append(answer)
        return "Ready to Play!"
    except Exception as e:
        return f"Training failed: {str(e)}"

def turn(audio_data, text_data):
    logging.debug(f'audio_data: {audio_data}, text_data: {text_data}')
    global questions, answers, score, current_question


    def check_answer(user_answer, correct_answer):
        rouge = evaluate.load('rouge')  # Carica ROUGE
        references = [correct_answer]  # Risposta corretta
        predictions = [user_answer]  # Risposta dell'utente

        results = rouge.compute(predictions=predictions, references=references)
        rouge_l_f1 = results['rougeL']  # F1 score per ROUGE-L

        # Restituisci il valore di somiglianza calcolato
        return rouge_l_f1

    if(audio_data is not None):
      user_answer = transcribe(audio_data)
    elif (text_data is not None):
      user_answer = str(text_data)
    else:
      user_answer = ""
    correct_answer_to_show = answers[current_question]

    if current_question <= len(questions):
        score = check_answer(user_answer, answers[current_question])
        current_question += 1
        return score, correct_answer_to_show
    else:
        return "","Game finished!"




def show_new_question():
    global current_question, questions
    if current_question >= len(questions):
        return "Game Finished!"
    return questions[current_question]

with gr.Blocks(theme=gr.themes.Soft()) as demo:

    with gr.Tab("Data Loading"):
        input_text = gr.Textbox(label = "input")
        btn = gr.Button("Upload text")
        show_status = gr.Textbox(label = "Show status")
        btn.click(fn=load_data, show_progress=True, inputs=input_text, outputs=[show_status])


    with gr.Tab("Game"):
        new_question_text = gr.Textbox(label=" ")
        audio = gr.Audio(type="filepath")
        text = gr.Textbox()
        submit_btn = gr.Button("Submit Answer")


        with gr.Row():
          score_text = gr.Label(label="Score")
          answer_correct_text = gr.Label(label="Correct Answer")
        submit_btn.click(turn, inputs=[audio, text], outputs=[score_text, answer_correct_text])

        new_question_btn = gr.Button("New Question")
        new_question_btn.click(show_new_question, outputs=new_question_text)


demo.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e34be711d821473c50.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
